https://mk.tula.ru/news/

In [43]:
import requests
import re
from bs4 import BeautifulSoup
import os.path

Следующие несколько функций собирают необходимую информацию о каждой статье:

In [44]:
#почему-то полного названия источника нет нигде на странице, оно находится абсолютно в другом разделе сайта
def get_source(soup):
    return 'Молодой коммунар'

In [45]:
calendar = {
    'Января':'01',
    'Февраля':'02',
    'Марта':'03',
    'Апреля':'04',
    'Мая':'05',
    'Июня':'06',
    'Июля':'07',
    'Августа':'08',
    'Сентября':'09',
    'Октября':'10',
    'Ноября':'11',
    'Декабря':'12'
}


def convert_date(s, calendar = calendar):
    date_split = s.split()[:3]
    date_split[1] = calendar[date_split[1]]
    return '.'.join(date_split)
    
def get_date(soup):
    date = soup.find('div', {'class': 'data_news'})
    date = date.find('span').string
    date = str(date)
    return convert_date(date)

In [46]:
#Авторы статей не указаны
def get_author(soup):
    return('na')

In [47]:
def get_title(soup):
    return soup.title.string

In [48]:
def clean_string(soup_obj):
    s = str(soup_obj)
    s = s.replace('\xa0', ' ')
    bs = BeautifulSoup(s)
    return(bs.get_text(strip=True))

def get_content(soup):
    content = soup.find('div', {'class': 'poster_content_block'})
    content = content.find_all('p')
    content_l = [el.get_text(strip=True) for el in content]

    return '\n'.join(content_l)

Функция, которая собирает все пункты информации:

In [49]:
def get_info(url, website = 'https://mk.tula.ru'):
    full_url = website + url
    
    response = requests.get(full_url)
    soup = BeautifulSoup(response.text.replace('\xa0', ' '))
    
    source = get_source(soup)
    date = get_date(soup)
    author = get_author(soup)
    title = get_title(soup)
    content = get_content(soup)
    
    res_l = ['=====', full_url, source, date, author, title, content]
    
    res = '\n'.join(res_l)
    return res

Функция, которая добавляет всю информацию об одной статье в файл:

In [50]:
def add_res_to_file(url, filename):
    info = get_info(url)
    
    if os.path.isfile(filename):
        with open(filename, 'a') as file:
            file.write(info + '\n')
    else:
        with open(filename, 'w') as file:
            file.write(info + '\n')


Функция, которая для каждой новости на странице добавляет информацию в файл

In [58]:
def add_all_to_file(page_url, filename, news_class = 'news_container_list_aimg'):
    response = requests.get(page_url)
    soup = BeautifulSoup(response.text)
    
    urls_set = set()
    for e in soup.find_all('a'):
        if 'class' in e.attrs:
            if news_class in e['class']:
                if e['href'] not in urls_set:
                    urls_set.add(e['href'])
                    add_res_to_file(e['href'], filename)
                    #print(e['href'])

In [59]:
#можно печатать страницы, чтобы понимать, что процесс идет и сколько работы уже выполнено

def all_pages(page_url, filename, end = 1453):
    for i in range(1, 21):
        add_all_to_file(page_url+'?PAGEN_2='+str(i), filename)
        print(page_url+'?PAGEN_2='+str(i))


In [60]:
all_pages('https://mk.tula.ru/news/', 'mktula.txt')
with open('mktula.txt', 'a') as file:
    file.write('=!====')

https://mk.tula.ru/news/?PAGEN_2=1
https://mk.tula.ru/news/?PAGEN_2=2
https://mk.tula.ru/news/?PAGEN_2=3
https://mk.tula.ru/news/?PAGEN_2=4
https://mk.tula.ru/news/?PAGEN_2=5
https://mk.tula.ru/news/?PAGEN_2=6
https://mk.tula.ru/news/?PAGEN_2=7
https://mk.tula.ru/news/?PAGEN_2=8
https://mk.tula.ru/news/?PAGEN_2=9
https://mk.tula.ru/news/?PAGEN_2=10
https://mk.tula.ru/news/?PAGEN_2=11
https://mk.tula.ru/news/?PAGEN_2=12
https://mk.tula.ru/news/?PAGEN_2=13
https://mk.tula.ru/news/?PAGEN_2=14
https://mk.tula.ru/news/?PAGEN_2=15
https://mk.tula.ru/news/?PAGEN_2=16
https://mk.tula.ru/news/?PAGEN_2=17
https://mk.tula.ru/news/?PAGEN_2=18
https://mk.tula.ru/news/?PAGEN_2=19
https://mk.tula.ru/news/?PAGEN_2=20
